In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import ipdb
import pandas as pd
from graphviz import Graph

In [8]:
dataframe = pd.read_csv("dataset.csv")
nb_people = len(dataframe.index)
print(dataframe.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            200 non-null    int64  
 1   age                   200 non-null    float64
 2   height                200 non-null    float64
 3   job                   200 non-null    object 
 4   city                  200 non-null    object 
 5   favorite music style  200 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 9.5+ KB
None


In [9]:
print(dataframe.columns)

Index(['Unnamed: 0', 'age', 'height', 'job', 'city', 'favorite music style'], dtype='object')


In [10]:
print(dataframe.head(10))

   Unnamed: 0        age      height         job       city   
0           0  30.237071  179.874298    designer      paris  \
1           1  27.915796  172.659587     fireman  marseille   
2           2  32.205338  181.337491     teacher      paris   
3           3  26.595215  172.337885    designer   toulouse   
4           4  27.394780  182.708030     teacher      paris   
5           5  29.710022  177.037196      doctor   toulouse   
6           6  41.789490  188.476525     fireman  marseille   
7           7  36.665622  175.102745     painter   toulouse   
8           8  33.838742  176.441942     painter      paris   
9           9  30.037158  191.462290  developper      paris   

  favorite music style  
0                 trap  
1               hiphop  
2                metal  
3                metal  
4                metal  
5                 rock  
6                  rap  
7                 rock  
8            classical  
9                 trap  


In [11]:
def compute_dissimilarity(id1, id2):
    entity_1_age = dataframe.loc[id1][1]
    entity_2_age = dataframe.loc[id2][1]

    entity_1_height = dataframe.loc[id1][2]
    entity_2_height = dataframe.loc[id2][2]

    entity_1_job = dataframe.loc[id1][3]
    entity_2_job = dataframe.loc[id2][3]

    entity_1_city = dataframe.loc[id1][4]
    entity_2_city = dataframe.loc[id2][4]

    entity_1_music_style = dataframe.loc[id1][5]
    entity_2_music_style = dataframe.loc[id2][5]

    if entity_1_job == entity_2_job:
        dissimilarity_job = 0
    else:
        dissimilarity_job = 15
    
    if entity_1_city == entity_2_city:
        dissimilarity_city = 0
    else:
        dissimilarity_city = 15

    if entity_1_music_style == entity_2_music_style:
        dissimilarity_music_style = 0
    elif entity_1_music_style == "technical death metal" and entity_2_music_style == "metal":
        dissimilarity_music_style = 5 
    else:
        dissimilarity_music_style= 15
    
    dissimilarity = math.sqrt(
        (entity_1_age - entity_2_age) ** 2
        + 3 * (entity_1_height - entity_2_height) ** 2
        + dissimilarity_job
        + dissimilarity_city
        + dissimilarity_music_style
    )

    print("==========")
    entity_1_id = dataframe.loc[id1][0]
    entity_2_id = dataframe.loc[id2][0]
    print(
        f"entity1 {entity_1_id}, entity_2_id {entity_2_id}, dissimilarity: {dissimilarity}"
    )
    return dissimilarity

In [12]:
#build dissimilarity matrix
dm = np.zeros((nb_people, nb_people))
print("compute dissimilarities...")
for entity_1_id in range(nb_people):
    for entity_2_id in range(nb_people):
        dissimilarity = compute_dissimilarity(entity_1_id, entity_2_id)
        dm[entity_1_id, entity_2_id] = dissimilarity
print("====== dissimilarity matrix ======")
print(dm)

compute dissimilarities...
entity1 0, entity_2_id 0, dissimilarity: 0.0
entity1 0, entity_2_id 1, dissimilarity: 14.371655635421424
entity1 0, entity_2_id 2, dissimilarity: 6.347981649652836
entity1 0, entity_2_id 3, dissimilarity: 14.6169660432081
entity1 0, entity_2_id 4, dissimilarity: 7.88471456638235
entity1 0, entity_2_id 5, dissimilarity: 8.332180026254692
entity1 0, entity_2_id 6, dissimilarity: 20.011329212319175
entity1 0, entity_2_id 7, dissimilarity: 12.435008966925137
entity1 0, entity_2_id 8, dissimilarity: 8.84958958936042
entity1 0, entity_2_id 9, dissimilarity: 20.4422258402945
entity1 0, entity_2_id 10, dissimilarity: 8.231717712538897
entity1 0, entity_2_id 11, dissimilarity: 7.640474170652596
entity1 0, entity_2_id 12, dissimilarity: 8.350237088793977
entity1 0, entity_2_id 13, dissimilarity: 5.9707008057904245
entity1 0, entity_2_id 14, dissimilarity: 6.723681388050649
entity1 0, entity_2_id 15, dissimilarity: 21.2126785983305
entity1 0, entity_2_id 16, dissimilari

In [24]:
threshold = 15

dot = Graph(comment="Graph created from dataset", strict=True)
for entity in range(nb_people):
    entity_id = dataframe.loc[entity][5]
    dot.node(entity_id)

for entity_1 in range(nb_people):
    # we use an undirected graph so we do not need
    # to take the potential reciprocal edge
    # into account
    for entity_2 in range(nb_people):
            # no self loops
        if not entity_1 == entity_2:
            entity_1_id = dataframe.loc[entity_1][0]
            entity_2_id = dataframe.loc[entity_2][0]
            # use the threshold condition
            # EDIT THIS LINE
            if dm[entity_1, entity_2] > threshold:
                dot.edge(
                    str(entity_1_id),
                    str(entity_2_id),
                    color="darkolivegreen4",
                    penwidth="1.1",
                )
# visualize the graph
dot.attr(label=f"threshold {threshold}", fontsize="20")
graph_name = f"images/complex_data_threshold_{threshold}"
dot.render(graph_name)